In [1]:
import pydeck as pyd
import pandas as pd

# Plotting lights at night

NASA has collected global light emission data for over 30 years. The data set is a deeply fascinating one and has been used for news stories on the Syrian Civil War [[1]](https://time.com/3741451/syria-lights-civil-war-satellite/), North Korea [[2]](https://www.cbsnews.com/pictures/north-korea-hermit-country-space-photos/4/), and economic growth [[3]](https://qz.com/959563/nasas-black-marble-map-shows-the-light-of-human-population-centers-at-night-throughout-2016/).

In this notebook, we'll use a deck.gl [ScreenGridLayer](https://deck.gl/#/examples/core-layers/screen-grid-layer) to visualize some of the changes at different points in time.

## Getting the data

The data for Chengdu, China, is cleaned and available below:

In [2]:
LIGHTS_URL = 'https://raw.githubusercontent.com/ajduberstein/lights_at_night/master/chengdu_lights_at_night.csv'
df = pd.read_csv(LIGHTS_URL)
df.head()

,year,lng,lat,brightness
0,1993,104.575,31.808,4
1,1993,104.583,31.808,4
2,1993,104.592,31.808,4
3,1993,104.600,31.808,4
4,1993,104.675,31.808,4


### Setting the colors
PyDeck does need to know the color for this data in advance of plotting it

In [3]:
df['color'] = df['brightness'].apply(lambda val: [255, val * 4,  255, 255])
df.sample(10)

,year,lng,lat,brightness,color
295953,2005,104.642,29.617,3,"[255, 12, 255, 255]"
61004,2009,105.392,31.217,21,"[255, 84, 255, 255]"
169694,2013,104.517,30.367,21,"[255, 84, 255, 255]"
208493,2007,103.158,30.142,3,"[255, 12, 255, 255]"
112543,2001,104.492,29.725,19,"[255, 76, 255, 255]"
158166,2013,103.442,30.983,7,"[255, 28, 255, 255]"
102570,2001,104.200,30.575,19,"[255, 76, 255, 255]"
95948,2001,103.683,30.958,11,"[255, 44, 255, 255]"
214575,2007,104.958,29.683,4,"[255, 16, 255, 255]"
212589,2007,102.808,29.842,3,"[255, 12, 255, 255]"


### Configuring the coordinates

Currently PyDeck expects coordinates to be an array listed in one field, which we can implement in Pandas:

In [4]:
df['position'] = df.apply(lambda row: [row['lng'], row['lat']], axis=1)
# Make the data frame smaller by only plotting useful fields
result_df = df[['position', 'color', 'year']]
result_df.head()

,position,color,year
0,"[104.575, 31.808000000000003]","[255, 16, 255, 255]",1993
1,"[104.583, 31.808000000000003]","[255, 16, 255, 255]",1993
2,"[104.592, 31.808000000000003]","[255, 16, 255, 255]",1993
3,"[104.6, 31.808000000000003]","[255, 16, 255, 255]",1993
4,"[104.675, 31.808000000000003]","[255, 16, 255, 255]",1993


## Plotting and interacting

We can plot this data set of light brightness by year, configuring a slider to filter the data as below:

In [5]:
plottable = result_df[result_df['year'] == 1993].to_dict(orient='records')

view_state = pyd.ViewState(latitude=31.0, longitude=104.5, zoom=8)
scatterplot = pyd.Layer(
    'ScatterplotLayer',
    data=plottable,
    get_position='position',
    get_fill_color='color',
    opacity=0.5,
    get_radius=800)
r = pyd.Deck(layers=[scatterplot], initial_view_state=view_state)
r.show()

DeckGLWidget(json_input='{"initialViewState": {"bearing": 0, "latitude": 31.0, "longitude": 104.5, "maxZoom": …

In [6]:
import ipywidgets as widgets
from IPython.display import display
slider = widgets.IntSlider(1992, min=1993, max=2013, step=2)
def on_change(v):
    plottable = result_df[result_df['year'] == slider.value].to_dict(orient='records')
    scatterplot.data = plottable
    r.update()
    
slider.observe(on_change, names='value')
display(slider)

IntSlider(value=1993, max=2013, min=1993, step=2)